# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import api_keys

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_path="../output_data/cities.csv"
city_df = pd.read_csv(weather_data_path)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tuktoyaktuk,69.4541,-133.0374,1.40,85,90,13.80,CA,1610376478
1,Ploemeur,47.7333,-3.4333,48.99,76,90,5.75,FR,1610376479
2,Rocha,-34.4833,-54.3333,85.77,52,100,6.82,UY,1610376479
3,Galle,6.0367,80.2170,77.67,86,99,5.68,LK,1610376449
4,Bredasdorp,-34.5322,20.0403,75.20,60,97,18.41,ZA,1610376479


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps
gmaps.configure(api_key = g_key)

# store longitude and Latitude
locs = city_df[['Lat','Lng']].astype(float)

# store humidity for the weights
humidity = city_df['Humidity'].astype(float)

In [4]:
# format the layout
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Create heatmap layer
fig = gmaps.figure(layout = figure_layout)

heat_layer = gmaps.heatmap_layer(locs, 
                              weights = humidity, 
                              dissipating = False, 
                              max_intensity = 100, 
                              point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# New data frame with the criteria
city_df_filter = city_df.loc[(city_df['Max Temp'] >= 70) 
                             & (city_df['Max Temp'] < 80)
                             & (city_df['Wind Speed'] < 10)
                             & (city_df['Cloudiness'] == 0)]
city_df_filter.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,Nelson Bay,-32.7167,152.1500,71.01,73,0,9.22,AU,1610376487
72,Akola,20.7333,77.0000,77.86,34,0,5.06,IN,1610376486
123,Beni Suef,29.0731,31.0979,72.57,25,0,6.42,EG,1610376690
209,Salalah,17.0151,54.0924,75.20,35,0,6.91,OM,1610376828
405,Craigieburn,-37.6000,144.9500,73.99,60,0,8.05,AU,1610377114


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# copy the filtered DF
hotel_df = city_df_filter.copy()
hotel_df["Hotel Name"]=""
city_limit = 10

# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
results = ""
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    params = {
    "radius": 5000, # radius
    "location": f"{row['Lat']},{row['Lng']}",
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key}

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        # get hotel name and replace the coordinates with actual hotel coordinates
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

# remove missing hotel names
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(inplace=True)

# apply city count limit
hotel_df = hotel_df.head(city_limit)

Closest hotel is Amarna Luxury Beach Resort - Guest Only.
------------
Closest hotel is Hotel Jasnagra.
------------
Closest hotel is Tolip Inn Beni Suef.
------------
Closest hotel is InterCityHotel Salalah.
------------
Missing field/result... skipping.
------------


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, info_box_content = hotel_info)

# Display figure
fig = gmaps.figure(layout = figure_layout)
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…